# Synthetic Charged Particle Radiographs with Multiple Detector Layers

[charged_particle_radiography]: ../../ad/diagnostics/charged_particle_radiography.rst

[Bragg peak]:
https://en.wikipedia.org/wiki/Bragg_peak
    
Charged particle radiographs are often recorded on detector stacks with multiple layers (often either radiochromic film or CR39 plastic). Since charged particles deposit most of their energy at a depth related to their energy (the [Bragg peak]), each detector layer records a different energy band of particles. The energy bands recorded on each layer are further discriminated by including layers of filter material (eg. thin strips of aluminum) between the active layers. In order to analyze this data, it is necessary to calculate what range of particle energies are stopped in each detector layer.

The [charged_particle_radiography] module includes a tool for calculating the energy bands for a given stack of detectors. 

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.data.data import get_file
from plasmapy.diagnostics import charged_particle_radiography as cpr

[charged_particle_radiography]: ../../ad/diagnostics/charged_particle_radiography.rst

[Layer]:
../../api/plasmapy.diagnostics.charged_particle_radiography.Layer.rst#plasmapy.diagnostics.charged_particle_radiography.Layer

[Stack]:
../../api/plasmapy.diagnostics.charged_particle_radiography.Stack.rst#plasmapy.diagnostics.charged_particle_radiography.Stack

[astropy Quantity]:
https://docs.astropy.org/en/stable/api/astropy.units.quantity.Quantity.html

[PSTAR]:
https://physics.nist.gov/PhysRefData/Star/Text/PSTAR.html

[ESTAR]:
https://physics.nist.gov/PhysRefData/Star/Text/ESTAR.html

The [charged_particle_radiography] module represents a detector stack with a [Stack] object, which contains an ordered list of [Layer] objects. In this example, we will create a list of [Layer] objects, use them to initialize a [Stack], then compute the energy bands.

Each [Layer] is defined by the following required properties:
- `thickness`: an [astropy Quantity] defining the thickness of the layer.
- `active`: a boolean flag which indicates whether the layer is active (detector) or inactive (eg. a filter).
- `energy_axis`: An [astropy Quantity] array of energies.
- `stopping_power`: An [astropy Quantity] array containing the product of stopping power and material density at each energy in `energy_axis`. 

The stopping power in various materials for protons and electrons is tabulated by NIST in the [PSTAR] and [ESTAR] databases. The stopping powers are tabulated in units of MeV cm$^2$ / g, so the product of stopping power and density has units of MeV/cm.

For this demonstration, we will load two stopping power tables, downloaded from [PSTAR], for aluminum and a human tissue equivalent that is a good approxmation for many radiochromic films.

In [ ]:
tissue_path = get_file("NIST_PSTAR_tissue_equivalent.txt")
aluminum_path = get_file("NIST_PSTAR_aluminum.txt")

arr = np.loadtxt(tissue_path, skiprows=8)
energy_axis = arr[:, 0] * u.MeV
tissue_density = 1.04 * u.g / u.cm ** 3
tissue_stopping_power = arr[:, 1] * u.MeV * u.cm ** 2 / u.g * tissue_density

arr = np.loadtxt(aluminum_path, skiprows=8)
aluminum_density = 2.7 * u.g / u.cm ** 3
aluminum_stopping_power = arr[:, 1] * u.MeV * u.cm ** 2 / u.g * aluminum_density

If we plot the stopping powers as a function of energy on a log-log scale, we see that they are non-linear, with much higher values at lower energies. This is why particles deposit most of their energy at a characteristic depth. As particles propagate into material, they lose energy slowly at first. But, as they loose energy, the effective stopping power increases expontentially. 

In [ ]:
fig, ax = plt.subplots()
ax.plot(energy_axis.value, tissue_stopping_power, label="Tissue equivalent")
ax.plot(energy_axis.value, aluminum_stopping_power, label="Aluminum")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Energy (MeV)")
ax.set_ylabel("Stopping power (MeV/cm)")
ax.legend();

We will start by creating a list that represents a common type of radiochromic film, [HDV2]. 

In [ ]:
HDV2 = [
    cpr.Layer(12 * u.um, energy_axis, tissue_stopping_power, name="2-HDV2-active"),
    cpr.Layer(
        97 * u.um,
        energy_axis,
        tissue_stopping_power,
        name="2-HDV2-substrate",
        active=False,
    ),
]